## 챗봇 예제(역할 부여)

In [1]:
# 사전 설치 : pip install ollama
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
import ollama
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever
import warnings

In [2]:
# 챗봇의 기본적 질문, 답변 역할
def ask_gemma(question):
    # ollama를 사용하여 모델로부터 응답 생성
    chatbot_role = "You are a helpful assistant."
    response = ollama.chat(model='gemma2', messages=[
        {"role": "system", "content": chatbot_role},  # 챗봇의 기본 역할 부여
        {"role": "user", "content": question}, # 질문
    ])

    return response['message']['content']

In [3]:
question = "세계에서 가장 높은 산을 알려줘."
response = ask_gemma(question)
print(response)

세계에서 가장 높은 산은 **에베레스트산**입니다. 해발 8,848.86m(29,031.7피트)로, 네팔과 중국 사이의 경계선 위에 위치해 있습니다.  


도움이 되셨기를 바랍니다! 😊



## 챗봇 예제(Gradio 사용)

In [4]:
# 사전 설치 : pip install gradio
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
import gradio as gr

In [5]:
# ChatOllama 모델 초기화
model = ChatOllama(model="gemma2", temperature=0.7, verbose=False)

C:\Users\human-14\AppData\Local\Temp\ipykernel_1732\3441953186.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="gemma2", temperature=0.7, verbose=False)


In [6]:
# 채팅 기록을 포함하여 응답을 생성하는 함수
def chat(message, history):
    # 이전 대화 기록을 ChatOllama 형식으로 변환
    chat_history = []
    for human, ai in history:
        chat_history.append(HumanMessage(content=human))
        chat_history.append(AIMessage(content=ai))

    # 현재 메시지 추가
    chat_history.append(HumanMessage(content=message))

    # 모델을 사용하여 응답 생성
    response = model.invoke(chat_history)

    return response.content

In [8]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "안녕하세요!",
        "인공지능에 대해 설명해주세요.",
        "파이썬의 장점은 무엇인가요?"
    ],
    title="AI 챗봇",
    description="질문을 입력하면 AI가 답변합니다."
)

c:\Ai_Project\.venv\lib\site-packages\gradio\components\chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [9]:
# 서버 실행
demo.launch(server_port=7861, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


In [10]:
demo.close()

Closing server running on port: 7861


## 챗봇 예제(Gradio + csv 사용)

In [11]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [13]:
# CSV 파일 로드
df = pd.read_csv("./dataset/indata_kor.csv", encoding='CP949')

In [14]:
# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text("\n".join(df.to_string()))

In [15]:
# 임베딩 모델 초기화
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\human-14\AppData\Local\Temp\ipykernel_1732\2819514520.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [16]:
# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)

In [17]:
# ChatOllama 모델 초기화
llm = ChatOllama(model="gemma2", tempeature=0.1)

In [18]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={"k":1}),
    return_source_documents=True,
    verbose=False
)

In [19]:
# 채팅 함수 정의
def chat(message, history):
    # 이전 대화 기록을 ConversationalRetrievalChain 형식으로 변환
    chat_history = [(human, ai) for human, ai in history]

    # 모델을 사용하여 응답 생성
    response = qa_chain({"question": message, "chat_history": chat_history})

    # 소스 문서 정보 추출
    sources = set([doc.metadata.get('source', 'Unknown') for doc in response['source_documents']])
    source_info = f"\n\n참고 출처: {', '.join(sources)}" if sources else ""

    return response['answer'] + source_info

In [20]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?",
        "스마트금융과에 대해 설명해주세요",
        "한국폴리텍대한 추천할만한 학과 하나를 소개해주세요."
    ],
    title="대학 정보 AI 챗봇",
    description="스마트금융과에 대한 질문을 입력하면 AI가 CSV데이터를 참고하여 한글로 답변합니다."
)

c:\Ai_Project\.venv\lib\site-packages\gradio\components\chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [21]:
# 서버 실행
demo.launch(server_port=7861, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\human-14\AppData\Local\Temp\ipykernel_1732\4160252103.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": message, "chat_history": chat_history})


In [22]:
demo.close()

Closing server running on port: 7861


## 챗봇 예제(인터넷 URL정보 요약하기)

In [151]:
import gradio as gr
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from wordcloud import WordCloud
from dotenv import load_dotenv
from gtts import gTTS
import tempfile
import whisper
import ollama
import matplotlib.pyplot as plt
import io

In [152]:
# load_dotenv()

False

In [153]:
os.environ["PATH"] += os.pathsep + r"C:/ffmpeg/bin"

In [154]:
def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"]

In [155]:
def process_audio(audio):
    if audio is None:
        return "오디오 파일을 업로드해주세요."
    try:
        transcribed_text = transcribe_audio(audio)
        return transcribed_text
    except Exception as e:
        return f"오류가 발생했습니다.: {str(e)}"

In [156]:
# Function to load, split, and retrieve documents

def load_and_retrieve_docs(url):
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict()
    )

    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    embeddings = OllamaEmbeddings(model="gemma2")

    # vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

    return vectorstore.as_retriever()

In [157]:
# Function to format documents

def format_docs(docs):

    return "\n\n".join(doc.page_content for doc in docs)

In [158]:
# Function that defines the RAG chain

def rag_chain(url, question):

    retriever = load_and_retrieve_docs(url)

    retrieved_docs = retriever.invoke(question)

    formatted_context = format_docs(retrieved_docs)

    formatted_prompt = f"Question: {question}\n\nContext: {formatted_context}"

    response = ollama.chat(model='gemma2', messages=[{'role': 'user', 'content': formatted_prompt}])

    return response['message']['content']

In [159]:
def generate_wordcloud(text):
    # 워드클라우드 생성
    wordcloud = WordCloud(
        font_path='malgun',
        background_color='white',
        width=800,
        height=600,
        max_words=200,
        max_font_size=100,
        min_font_size=10,
        random_state=42
    ).generate(text)
    
    # 플롯 생성 및 figure 객체 반환
    fig = plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    return fig

In [27]:
# # Gradio interface

# iface = gr.Interface(

#     fn=rag_chain,

#     inputs=["text", "text"],

#     outputs="text",

#     title="RAG Chain Question Answering",

#     description="Enter a URL and a query to get answers from the RAG chain."

# )

In [28]:
# # 디버그 모드로 Gradio 인터페이스 실행
# iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\human-14\AppData\Local\Temp\ipykernel_1732\2629661401.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma2")


Keyboard interruption in main thread... closing server.


In [160]:
# Function to get text from URL
def get_text_from_url(url):
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict()
    )
    docs = loader.load()
    return " ".join([doc.page_content for doc in docs])

In [161]:
def text_to_speech(text, lang='ko'):
    # 임시 파일 생성
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
        temp_filename = fp.name

    # TTS 변환
    tts = gTTS(text=text, lang=lang)
    tts.save(temp_filename)
    return temp_filename

In [162]:
def process_tts(text, lang):
    if not text:
        return None, "텍스트를 입력해주세요."
    try:
        audio_file = text_to_speech(text, lang)
        return audio_file, "변환이 완료되었습니다. 아래에서 재생 또는 다운로드할 수 있습니다."
    except Exception as e:
        return None, f"오류가 발생했습니다: {str(e)}"

In [165]:
# Gradio Tabbed Interface
with gr.Blocks() as iface:
    # Tab for Question and Answer
    with gr.Tab("질문과 답변"):
        gr.Interface(
            fn=rag_chain,
            inputs=["text", "text"],
            outputs="text",
            title="RAG Chain Question Answering",
            description="Enter a URL and a query to get answers from the RAG chain."
        ).render()

    with gr.Tab("시각화 (워드클라우드)"):
        with gr.Row():
            text_input = gr.Textbox(
                label="URL 또는 텍스트 입력",
                placeholder="URL이나 텍스트를 입력하세요",
                lines=5
            )
        
        def process_input(input_text):
            if input_text.startswith(('http://', 'https://')):
                text = get_text_from_url(input_text)
            else:
                text = input_text
            return generate_wordcloud(text)
        
        plot_output = gr.Plot()  # Plot 컴포넌트 사용
        generate_btn = gr.Button("워드클라우드 생성")
        
        generate_btn.click(
            fn=process_input,
            inputs=text_input,
            outputs=plot_output
        )
    with gr.Tab("음성을 텍스트로 변환"):
        gr.Interface(
            fn=process_audio,
            inputs=gr.Audio(type="filepath", label="MP3 파일 업로드"),
            outputs = "text",
            title = "MP3 to Text Converter",
            description = "MP3 파일을 업로드하면 텍스트로 변환합니다."
        )
    with gr.Tab("텍스트를 음성으로 변환"):
        gr.Interface(
            fn=process_tts,
            inputs=[
                gr.Textbox(lines=5, label="텍스트 입력"),
                gr.Dropdown(choices=['ko','en','ja','zh-cn'], label="언어 선택", value='ko')
            ],
            outputs=[
                gr.Audio(label="생성된 오디오"),
                gr.Textbox(label="상태 메시지")
            ],
            title = "Text to Speech Converter",
            description = "텍스트를 입력하면 MP3파일로 변환합니다."
        )

In [168]:
# 디버그 모드로 Gradio 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [169]:
iface.close()

Closing server running on port: 7861


## 챗봇 예제(STT:음성을 텍스트로 전환)

In [61]:
import os
from dotenv import load_dotenv
import whisper
import gradio as gr

In [62]:
load_dotenv()

False

In [69]:
# ffmpeg 경로 명시적 설정
# os.environ["FFMPEG_BINARY"] = "C:/aiproject/ffmpeg/bin/ffmpeg.exe"
os.environ["PATH"] += os.pathsep + r"C:/ffmpeg/bin"
# os.environ["FFMPEG_BINARY"] = r"C:/Ai_Project/ffmpeg/bin/ffmpeg.exe"

In [70]:
def transcribe_audio(audio_path):
    # Whisper 모델 로드
    model = whisper.load_model("base")
    
    # 오디오 파일 전사
    result = model.transcribe(audio_path)
    
    # 전사된 텍스트 반환
    return result["text"]

In [71]:
def process_audio(audio):
    if audio is None:
        return "오디오 파일을 업로드해주세요."
    try:
        transcribed_text = transcribe_audio(audio)
        return transcribed_text
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

In [72]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=process_audio, 
    inputs = gr.Audio(type="filepath", label="MP3 파일 업로드"),
    outputs="text", 
    title = "MP3 to Text Converter",
    description = "MP3 파일을 업로드하면 텍스트로 변환합니다."
)

In [73]:
# 디버그 모드로 Gradio 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


c:\Ai_Project\.venv\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
c:\Ai_Project\.

Keyboard interruption in main thread... closing server.


In [74]:
iface.close()

Closing server running on port: 7861


## 챗봇 예제(TTS:텍스트를 음성변환)

In [124]:
# 사전 설치 : pip install gtts
import gradio as gr
from gtts import gTTS
import os
import tempfile

In [125]:
def text_to_speech(text, lang='ko'):
    # 임시 파일 생성
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
        temp_filename = fp.name
    
    # TTS 변환
    tts = gTTS(text=text, lang=lang)
    tts.save(temp_filename)
    return temp_filename

In [126]:
def process_tts(text, lang):
    if not text:
        return None, "텍스트를 입력해주세요."
    try:
        audio_file = text_to_speech(text, lang)
        return audio_file, "변환이 완료되었습니다. 아래에서 재생 또는 다운로드할 수 있습니다."
    except Exception as e:
        return None, f"오류가 발생했습니다.: {str(e)}"

In [127]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=process_tts, 
    inputs = [
        gr.Textbox(lines=5, label="텍스트 입력"),
        gr.Dropdown(choices=['ko','en','ja','zh-cn'], label="언어 선택", value='ko')
    ], 
    outputs=[
        gr.Audio(label="생선된 오디오"),
        gr.Textbox(label="상태 메시지")
    ],
    title="Text to Speech Converter",
    description="텍스트를 입력하면 MP3 파일로 변환합니다."
)

In [128]:
# 디버그 모드로 Gradio 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv
Keyboard interruption in main thread... closing server.


In [129]:
iface.close()

Closing server running on port: 7861
